In [1]:
import lammps_logfile
import scipy
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import PySimpleGUI as sg
import numpy as np
import PySimpleGUI as sg
import matplotlib.widgets as wg
import matplotlib
import matplotlib.pyplot as plt

sg.theme("DarkBlue7")
color1 = sg.theme_input_background_color()
color = sg.theme_button_color_background()

In [10]:
# Method used only once to create the main window of the program.
def make_main_window():
    layout = [
        [sg.FilesBrowse('Load Log Files...',
                        enable_events=True, key='file_main',
                        file_types=(('LAMMPS Log Files', 'log.*'),
                                    ('LAMMPS Log Files', '*.lammps'),
                                    ('LAMMPS Log Files', '*.out')))],
        [sg.TabGroup([[]], key='tab_group_main', expand_x=True, expand_y=True)]

    ]
    return sg.Window('MDGlass viewer', layout, resizable=True, finalize=True)

In [2]:
# Method used to create new tabs in the main window for each loaded file.
def make_new_tab_main(tab_name, tab_id):
    frame_layout = []
    for i in range(1, 5):
        frame_layout.append([[sg.T('N:'), sg.Combo([], key=f'combo_n{i}_{tab_id}', expand_x=True),
                             sg.T('X:'), sg.Combo(
                                 [], key=f'combo_x{i}_{tab_id}', expand_x=True),
                             sg.T('Y:'), sg.Combo([], key=f'combo_y{i}_{tab_id}', expand_x=True)]])

    layout = [
        [sg.Canvas(key=f'controls_cv_{tab_id}'),
         sg.Push(),
         sg.B('Plot', key=f'plot_main_{tab_id}'), sg.B(
            'Analyze', key=f'analyze_{tab_id}'), sg.B('Close Tab', key=f'close_tab_{tab_id}')
         ],
        [sg.Column(
            layout=[
                [sg.Frame('', frame_layout[0], expand_x=True),
                 sg.Frame('', frame_layout[1], expand_x=True)],
                [sg.Frame('', frame_layout[2], expand_x=True),
                 sg.Frame('', frame_layout[3], expand_x=True)],
                [sg.Canvas(key=f'fig_cv_{tab_id}',
                           size=(600 * 2, 600), expand_x=True, expand_y=True
                           )]

            ],
            pad=(0, 0),
            expand_x=True, expand_y=True
        )]
    ]

    return sg.Tab(tab_name, layout=layout, expand_x=True, expand_y=True, key=tab_id)

In [5]:
# Method used to make analyze windows for each loaded log file
def make_analyze_window(name, win_id):
    rdf_layout = [
        [sg.Canvas(key=f'controls_cv_rdf_{win_id}'), sg.Push(),
         sg.FilesBrowse('Load RDF Files...',
                        enable_events=True, key=f'file_rdf_{win_id}',
                        file_types=(('LAMMPS RDF Files', '*.rdf'),),
                        target=(0, 2)),
         sg.B('Clear RDF Data', key=f'clear_rdf_data_{win_id}', disabled=True),
         sg.B('Plot', key=f'plot_rdf_{win_id}'),
         sg.Input(key=f'rdf_xy_{win_id}', disabled=True, use_readonly_for_disable=True,
                  disabled_readonly_background_color=color1, size=(20, 0))
         ],
        [sg.T('N:'), sg.Combo([], key=f'combo_nrdf_{win_id}', expand_x=True),
            sg.T('X:'), sg.Combo(
                [], key=f'combo_xrdf_{win_id}', expand_x=True),
            sg.T('Y:'), sg.Combo(
                [], key=f'combo_yrdf_{win_id}', expand_x=True),
            sg.T('RDF Col:'), sg.Combo(
                [], key=f'combo_colrdf_{win_id}', expand_x=True, disabled=True)
         ],
        [sg.Canvas(key=f'fig_cv_rdf_{win_id}',
                       # it's important that you set this size
                       size=(600 * 2, 600), expand_x=True, expand_y=True
                   )]
    ]

    fit_headings = ['Line', 'Slope', 'Intercept', 'Boundry 1', 'Boundry 2']
    table = sg.Table(values=[], headings=fit_headings,
                     enable_events=True,
                     enable_click_events=True,
                     expand_x=True,
                     expand_y=True,
                     select_mode='extended',
                     key=f'table_fit_{win_id}')

    fit_layout = [
        [sg.Canvas(key=f'controls_cv_fit_{win_id}'), sg.Push(),
         sg.B('Plot', key=f'plot_fit_{win_id}'),
         sg.B('Intersection point', key=f'intersection_{win_id}'),
         sg.Input(key=f'intersection_value_{win_id}', disabled=True, use_readonly_for_disable=True,
                  disabled_readonly_background_color=color, size=(20, 0))
         ],
        [sg.T('N:'), sg.Combo([], key=f'combo_nfit_{win_id}', expand_x=True),
            sg.T('X:'), sg.Combo(
                [], key=f'combo_xfit_{win_id}', expand_x=True),
            sg.T('Y:'), sg.Combo(
                [], key=f'combo_yfit_{win_id}', expand_x=True)],
        [sg.Canvas(key=f'fig_cv_fit_{win_id}',
                       # it's important that you set this size
                       size=(600 * 2, 600), expand_x=True, expand_y=True
                   )],
        [table]
    ]
    
    view_layout = [
        [sg.Canvas(key=f'controls_cv_view_{win_id}'), sg.Push(),
         sg.FilesBrowse('Load Dump Files...',
                        enable_events=True, key=f'file_view_{win_id}',
                        file_types=(('LAMMPS dump Files', '*.dump'),),
                        target=(0, 2)),
         sg.B('Clear Dump Data', key=f'clear_dump_data_{win_id}', disabled=True),
         sg.B('Plot', key=f'plot_view_{win_id}')
         ],
        [sg.T('N:'), sg.Combo([], key=f'combo_nview_{win_id}', expand_x=True),
            sg.T('X:'), sg.Combo(
                [], key=f'combo_xview_{win_id}', expand_x=True),
            sg.T('Y:'), sg.Combo(
                [], key=f'combo_yview_{win_id}', expand_x=True)],
        [sg.Canvas(key=f'fig_cv_view_{win_id}',
                       # it's important that you set this size
                       size=(600 * 2, 600), expand_x=True, expand_y=True)]
    ]

    layout = [
        [sg.TabGroup([[sg.Tab('Linear Fit', layout=fit_layout, expand_x=True, expand_y=True, key=f'fit_{win_id}'),
                       sg.Tab('RDF', layout=rdf_layout, expand_x=True, expand_y=True, key=f'rdf_{win_id}'),
                       sg.Tab('View System', layout=view_layout, expand_x=True, expand_y=True, key=f'view_{win_id}')
                      ]],
                     key=f'tab_group_analyze_{win_id}', expand_x=True, expand_y=True)]
    ]

    return sg.Window(name, layout, resizable=True, finalize=True)